In [28]:
import paramiko
import sys
from os.path import expanduser
import time
import json

In [5]:
def connect():
    server = 'ec2-35-166-134-236.us-west-2.compute.amazonaws.com'
    k = paramiko.RSAKey.from_private_key_file('/Users/timlee/Dropbox/keys/chaffixdevkey.pem')
    c = paramiko.SSHClient()
    c.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    print "connecting"
    c.connect(hostname = server, username = 'ec2-user')
    return c

In [185]:
def get_timestamp():
    return time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())


def check_log(client):
    msg = 'opened connection at %s' % get_timestamp()    
    while True:
        _, stdout, _ = client.exec_command('cat flock_log.log | wc -l')
        line_count = int(stdout.read().split('\n')[0])
        if line_count % 2 == 0:
            print "%d Logs found" % line_count
            client.exec_command("echo %s > flock_log.log" % msg )
            break
        print "..."
        time.sleep(10)
    print "no other connections"

def git_pull(client):
    try:
        client.exec_command("cd ~/; git clone https://github.com/ajhoward7/sprint1.git")
        print 'repository created'
    except Exception as e:
        print e
        
def update_crontab(client):
    msg = '* * * * * python /home/ec2-user/sprint1/json_digest.py'
    try:
        client.exec_command('crontab -r')
        #client.exec_command('crontab -l | {cat; echo "%s"; } | crontab -' % msg)
        client.exec_command('(crontab -l 2>/dev/null; echo "*/5 * * * * python /home/ec2-user/sprint1/json_digest.py --prefix ppp") | crontab -')
        print 'cron tab updated'
    except Exception as e:
        print(e)
        
    

In [187]:
c = connect()
git_pull(c)
update_crontab(c)

connecting
repository created
cron tab updated


In [165]:
git_pull(c)

repository created
